In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, avg, max, length, mean, regexp_replace

In [2]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [3]:
df_titles = spark.read.csv(r'C:\Users\lucas\Downloads\EDC-modulo-2\trabalho pratico\dados\title_basics.tsv', header=True,
inferSchema=True, sep='\t')

In [4]:
df_ratings = spark.read.csv(r'C:\Users\lucas\Downloads\EDC-modulo-2\trabalho pratico\dados\title_ratings.tsv', header=True,
inferSchema=True, sep='\t')

In [32]:
# df_ratings.show()
# df_titles.show()

In [6]:
df_join = df_ratings.join(df_titles, ['tconst'], 'outer')

In [7]:
df_join.createTempView("filmes")

In [8]:
#Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
ano_2015 = spark.sql('SELECT COUNT(primaryTitle) AS count_2015 FROM filmes WHERE startYear = 2015')

In [9]:
ano_2015.show()

+----------+
|count_2015|
+----------+
|    358054|
+----------+



In [10]:
genero_frequente = df_join.groupBy("genres").count().orderBy(("count")).first()[0]

In [12]:
df = df_join.withColumn('filme', split(df_join['genres'], ',').getItem(0))

In [13]:
df = df.withColumn("genres", regexp_replace("genres", "Short", ""))

In [14]:
df = df.withColumn("filme", regexp_replace("filme", "Short", ""))

In [15]:
df = df.withColumn("genres", regexp_replace("genres", ",", ""))

In [16]:
from pyspark.sql.functions import desc, col, count, first

In [17]:
#Qual o gênero de títulos mais frequente?
genero_mais_frequente = df.groupBy("genres")\
                         .agg(count("*").alias("frequencia"))\
                         .orderBy(desc("frequencia"))\
                         .select(first("genres"))\
                         .head()[0]

In [18]:
genero_mais_frequente

'Drama'

In [19]:
df_filtered = df.filter((df.startYear == 2020))

In [20]:
df_grouped = df_filtered.groupBy('primaryTitle', 'titleType').agg(avg('averageRating').alias('avg_rating'))

In [21]:
df_result = df_grouped.select('primaryTitle', 'titleType', 'avg_rating').orderBy(desc('avg_rating')).limit(1)

In [22]:
#Quantos filmes têm o título atual (“primary”) diferente do título original?
df.select('primaryTitle').distinct().count()

3931670

In [23]:
#Quantos filmes têm o título atual (“primary”) diferente do título original?
different_title_count = df_titles.filter(df_titles['primaryTitle'] != df_titles['originalTitle']).count()

In [24]:
different_title_count

125056

In [25]:
# Qual o filme que tem o nome mais longo?
df_longest_title = df_titles.orderBy(length("primaryTitle").desc())
longest_title = df_longest_title.first()["tconst"]

In [26]:
longest_title

'tt12985206'

In [27]:
#Qual filme tem a maior quantidade de votos?
mais_votado = df.orderBy('numVotes', ascending=False).first()

In [28]:
#Qual filme tem a maior quantidade de votos? 
print(mais_votado['tconst'])

tt0111161


In [29]:
#Qual é a menor nota média de um filme?
df.select('averageRating').describe().filter('summary == "min"').show()

+-------+-------------+
|summary|averageRating|
+-------+-------------+
|    min|          1.0|
+-------+-------------+



In [30]:
#Qual a duração média dos filmes com conteúdo adulto?
# Filtrando os filmes com conteúdo adulto
df_adult = df.filter(col("isAdult") == 1)

# Calculando a duração média desses filmes
df_adult.select("runtimeMinutes").describe().show()

+-------+-----------------+
|summary|   runtimeMinutes|
+-------+-----------------+
|  count|           250127|
|   mean|92.79938555059914|
| stddev|57.18982244754776|
|    min|                1|
|    max|               \N|
+-------+-----------------+

